In [ ]:
!pip install bs4

In [ ]:
from bs4 import BeautifulSoup
import requests

cont = {}
for i in range(2):
  headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"}
  url = f'https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=list&&articleLimit=10&article.offset={i*10}'
  res = requests.get(url, headers=headers)
  soup = BeautifulSoup(res.text, 'lxml')

  contents = soup.select('tbody > tr > td.title.left > a')
  for i in contents:
    cont[i.get("title")] = 'https://www.kumoh.ac.kr/ko/sub06_01_01_01.do'+ i.get("href")
print(cont)


{'2023학년도 2학기 등록금 납부 안내(최종: 2023.9.25.~26.)': 'https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=463875&article.offset=10&articleLimit=10', '수업 중 실시간  출결 처리 시 전자출결 앱 사용 유의사항 안내': 'https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=462992&article.offset=10&articleLimit=10', '10/2(월,임시공휴일) 휴강 및 국가공휴일 휴강에 따른 보강일(공휴일지정보강기간) 안내': 'https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=461678&article.offset=10&articleLimit=10', '2023학년도 2학기 취업진로상담 교과목 시행안내': 'https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=461550&article.offset=10&articleLimit=10', '★ 2023학년도 2학기 휴학/복학 신청(3차) 안내 및 직권제적/휴학연장 관련 안내': 'https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=457290&article.offset=10&articleLimit=10', '2023-2학기 공결신청 방법 및 유의사항 안내': 'https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=view&articleNo=456000&article.offset=10&articleLimit=10', '[재공지][학생성공처]2023학년도 KIT역량장학금(K-WEGO) 안내': 'https://www.kumoh.ac.kr/ko/sub06_01_01_01.do?mode=vi

In [ ]:
import re

sen_list = []
i = 0
count = len(cont)
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"}
for link in cont.values():
  print(f"크롤링 중({count-i}개)",end = " ")
  i += 1
  r = requests.get(link, headers=headers)
  s = BeautifulSoup(r.text, 'lxml')

  c = s.select("#jwxe_main_content > div.contents-wrapper > div.board-area.ko.board.view > div.board-wrap > div > div.board-contents > p > span")
  le = ""
  for word in c:
    le += word.text.strip()
  sen_list.append(re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s().,-/:~""'']", " ", le))

크롤링 중(30개) 크롤링 중(29개) 크롤링 중(28개) 크롤링 중(27개) 크롤링 중(26개) 크롤링 중(25개) 크롤링 중(24개) 크롤링 중(23개) 크롤링 중(22개) 크롤링 중(21개) 크롤링 중(20개) 크롤링 중(19개) 크롤링 중(18개) 크롤링 중(17개) 크롤링 중(16개) 크롤링 중(15개) 크롤링 중(14개) 크롤링 중(13개) 크롤링 중(12개) 크롤링 중(11개) 크롤링 중(10개) 크롤링 중(9개) 크롤링 중(8개) 크롤링 중(7개) 크롤링 중(6개) 크롤링 중(5개) 크롤링 중(4개) 크롤링 중(3개) 크롤링 중(2개) 크롤링 중(1개) 

In [ ]:
sf_list = []
for i in sen_list:
  a = list(i.split("."))
  for i in range(len(a)):
    a[i] = a[i] + "\n"
  sf_list.append(a)


In [ ]:
crawling = dict(zip(cont.keys(),sf_list))

In [ ]:
"""del_k = []
for title, text in crawling.items():
  if text == ['']:
    del_k.append(title)
for i in del_k:
  del crawling[i]"""

"del_k = []\nfor title, text in crawling.items():\n  if text == ['']:\n    del_k.append(title)\nfor i in del_k:\n  del crawling[i]"

In [ ]:
#구글드라이브 마운트하기
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
for title, text in crawling.items():
  title = title.replace("/","-")
  if text == ['']:
    continue
  path = f'/content/drive/My Drive/Colab Notebooks/data/kit_text/{title}.txt'
  with open(path,'w') as f:
    for line in text:
      f.write(line)

In [ ]:
!pip install langchain unstructured openai chromadb Cython tiktoken pypdf

In [ ]:
!pip install sentence-transformers

In [ ]:
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI

from langchain.embeddings import LlamaCppEmbeddings, HuggingFaceEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings

In [ ]:
fn_dir = "/content/drive/My Drive/Colab Notebooks/data/kit_text"

In [ ]:
loader = DirectoryLoader(fn_dir)
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

len(documents), len(docs)

(110, 103)

In [ ]:
"""doc = docs
a = ""
w_count = 0
doc_count = 0
source_list = []
for i in doc:
  if list(i)[1][1] in source_list:
    a += list(i)[0][1]
    doc
  else :
    if w_count != 0:
      list(doc[doc_count])[0][1] = a
    a = ""
    a += list(i)[0][1]
    source_list.append(list(i)[1][1])
    doc_count = w_count
  w_count += 1
print(doc)"""

'doc = docs\na = ""\nw_count = 0\ndoc_count = 0\nsource_list = []\nfor i in doc:\n  if list(i)[1][1] in source_list:\n    a += list(i)[0][1]\n    doc\n  else :\n    if w_count != 0:\n      list(doc[doc_count])[0][1] = a\n    a = ""\n    a += list(i)[0][1]\n    source_list.append(list(i)[1][1])\n    doc_count = w_count\n  w_count += 1\nprint(doc)'

In [ ]:
"""a = ""
source_list = []
text_list = []
for i in docs:
  if list(i)[1][1] in source_list:
    a += list(i)[0][1]
  else :
    a = ""
    a += list(i)[0][1]
  text_list.append(a)
  source_list.append(list(i)[1][1])
print(text_list)"""

'a = ""\nsource_list = []\ntext_list = []\nfor i in docs:\n  if list(i)[1][1] in source_list:\n    a += list(i)[0][1]\n  else :\n    a = ""\n    a += list(i)[0][1]\n  text_list.append(a)\n  source_list.append(list(i)[1][1])\nprint(text_list)'

In [ ]:
db = Chroma.from_documents(docs, embedding=embeddings,
                           persist_directory="school_index_hf")
db.persist()

TypeError: ignored

In [ ]:
query = "장학생"
docs = db.similarity_search(query)

In [ ]:
docs[:5]